In [1]:
import numpy as np
import torch
import torch.nn as nn
from transformers import BertTokenizer
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data import RandomSampler, SequentialSampler
from torch import optim
from transformers import BertForSequenceClassification



c:\ProgramData\anaconda3\envs\Nlp_38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"


model = BertForSequenceClassification.from_pretrained(
    pretrained_model_name_or_path="bert-base-multilingual-cased", num_labels=2
).to(device)
optimizer = optim.AdamW(model.parameters(), lr=1e-5, eps=1e-8)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
model.load_state_dict('./BERTSequenceClassification.pt')

TypeError: Expected state_dict to be dict-like, got <class 'str'>.

In [4]:
print(type(model.state_dict))

<class 'method'>


In [6]:
model_state_dict = torch.load('./BERTSequenceClassification.pt', map_location=device)
model.load_state_dict(model_state_dict)

<All keys matched successfully>

In [7]:
test_text = '공공기관에 와서 무조건 우기고 소리지르면 다 되는줄 아는 민원인들 처벌하는 법이 있어야 한다. 왜이렇게 비이성적인 사람들을 상대하느라 다른 멀쩡한 사람들이 자꾸 피해를 봐야 하나'

In [14]:
tokenizer = BertTokenizer.from_pretrained(
    pretrained_model_name_or_path="bert-base-multilingual-cased", do_lower_case=False
)

In [16]:
tokenized = tokenizer(text=[test_text],padding="longest",truncation=True, return_tensors='pt')
tokenized

{'input_ids': tensor([[   101,   8896,  28000,  12310,  20595,  10530,   9590,  12424,   9294,
          20626,  71439,   9604,  12310,  11664,   9448,  12692,  12508,  31401,
          14867,   9056,  54780, 119219,   9519,  11018,   9311,  14279,  12030,
          27023,   9744,  68773,  12178,   9341,  10739,  45893,  21711,  16139,
            119,   9596,  10739,  82838,   9379,  10739,  17138,  15387,   9405,
          61250,  25820,   9414,  14423,  35506, 118760,  17342,  19709,    100,
           9405,  61250,  20173,   9651, 118694,   9946,  92278,   9363,  21711,
           9952,  16439,    102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [39]:
model(tokenized['input_ids']).logits

tensor([[ 1.4638, -1.3961]], grad_fn=<AddmmBackward0>)

In [41]:
model(tokenized['input_ids']).logits.squeeze().tolist()

[1.4638495445251465, -1.3960636854171753]

In [49]:
max(torch.sigmoid(model(tokenized['input_ids']).logits).squeeze().tolist())

0.8121207356452942

In [50]:
def plusminus(test_text):
    tokenized = tokenizer(text=[test_text],padding="longest",truncation=True, return_tensors='pt')
    resultList=torch.sigmoid(model(tokenized['input_ids']).logits).squeeze().tolist()
    result = resultList.index(max(resultList))
    if result == 0:
        print(f'부정적인 의미일 확률 : {max(resultList)}')
    else:
        print(f'긍정적인 의미일 확률 : {max(resultList)}')

In [51]:
plusminus(test_text=test_text)

부정적인 의미일 확률 : 0.8121207356452942


In [52]:
test_num2='악쓰고 터무니 없는 억지 주장으로 행패부리는 것들이 더럽게 많은 현실.그걸 또 맞상대하지말고 무조건 참으라고만 하는 윗선.'

In [53]:
plusminus(test_num2)

부정적인 의미일 확률 : 0.6718650460243225


In [54]:
test_good='좋은 행사네요 똑쟁이강아지 좋은 가정 가서 행복하게 살기를'
plusminus(test_good)

긍정적인 의미일 확률 : 0.9095252752304077


In [55]:
test_jy = """중소기업 취업자 가운데 청년층 비중이 대기업과 큰 격차를 보이는 것은 그만큼 청년층의 중소기업 기피 현상이 심화했기 때문이다.

중소기업 근로자 임금은 대기업과 2배 이상 차이를 보이고 육아휴직 등 근로 조건도 열악한 편이다.

전문가들은 중소기업의 다양한 성공 사례를 통해 청년에게 중소기업 미래에 대한 비전을 제시하고 일·가정 양립을 위한 사업주의 노력과 정부의 적극적인 지원이 필요하다고 주문했다."""

In [56]:
plusminus(test_jy)

부정적인 의미일 확률 : 0.693778395652771
